In [11]:
import torch
from torch_geometric.datasets import ModelNet
from torch_geometric.transforms import SamplePoints, NormalizeScale
from torch_geometric.loader import DataLoader

import open3d as o3d
import plotly.graph_objects as go

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import datetime
import os
import random

In [12]:
from pointnet import PointNetClassifier, PointNetClassificationLoss

In [13]:
device = "cuda"

In [64]:
# modelnet10

num_points = 1024

pre_transform = NormalizeScale()
transform = SamplePoints(num_points)

batch_size = 64

root = 'data/ModelNet10'
dataset_train = ModelNet(root=root, name='10', train=True, pre_transform=pre_transform, transform=transform)
trainloader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)

dataset_test = ModelNet(root=root, name='10', train=False, pre_transform=pre_transform, transform=transform)
testloader = DataLoader(dataset_test, batch_size=batch_size)

print(f'Number of training examples: {len(dataset_train)}')
print(f'Number of test examples: {len(dataset_test)}')

classes = dataset_test.raw_file_names
print(classes)

data = dataset_train[1001]
print(data)
print(f'Point cloud shape: {data.pos.shape}')
print(f'Label: {data.y}')

Number of training examples: 3991
Number of test examples: 908
['bathtub', 'bed', 'chair', 'desk', 'dresser', 'monitor', 'night_stand', 'sofa', 'table', 'toilet']
Data(pos=[1024, 3], y=[1])
Point cloud shape: torch.Size([1024, 3])
Label: tensor([2])


In [14]:
# modelnet40

num_points = 512

pre_transform = NormalizeScale()
transform = SamplePoints(num_points)

batch_size = 64

root = 'data/ModelNet40'
dataset_train = ModelNet(root=root, name='40', train=True, pre_transform=pre_transform, transform=transform)
trainloader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)

dataset_test = ModelNet(root=root, name='40', train=False, pre_transform=pre_transform, transform=transform)
testloader = DataLoader(dataset_test, batch_size=batch_size)

classes = ["airplane", "bathtub", "bed", "bench", "bookshelf", "bottle", "bowl", "car", "chair", "cone", "cup", "curtain", "desk", "door", "dresser", "flower_pot", "glass_box", "guitar", "keyboard", "lamp", "laptop", "mantel", "monitor", "night_stand", "person", "piano", "plant", "radio", "range_hood", "sink", "sofa", "stairs", "stool", "table", "tent", "toilet", "tv_stand", "vase", "wardrobe", "xbox"]

print(f'Number of training examples: {len(dataset_train)}')
print(f'Number of test examples: {len(dataset_test)}')

data = dataset_train[1001]
print(data)
print(f'Point cloud shape: {data.pos.shape}')
print(f'Label: {data.y}')

Number of training examples: 9843
Number of test examples: 2468
Data(pos=[512, 3], y=[1])
Point cloud shape: torch.Size([512, 3])
Label: tensor([2])


In [15]:
fig = go.Figure(
  data=[
    go.Scatter3d(
      x=data.pos[:,0], y=data.pos[:,1], z=data.pos[:,2],
      mode='markers',
      marker=dict(size=1, color="white"))],
  layout=dict(
    scene=dict(
      xaxis=dict(visible=False),
      yaxis=dict(visible=False),
      zaxis=dict(visible=False))))

fig.update_layout(template='plotly_dark')

fig.show()

In [60]:
pointnet = PointNetClassifier(num_classes=10)
pointnet.to(device)

# rotate object around z-axis
# apply guassian noise to each point, mean 0 std deviation 0.002

PointNetClassifier(
  (backbone): PointNetBackbone(
    (tnet1): Transformer(
      (mlp): Sequential(
        (0): Conv1d(3, 64, kernel_size=(1,), stride=(1,))
        (1): ReLU()
        (2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (3): Conv1d(64, 128, kernel_size=(1,), stride=(1,))
        (4): ReLU()
        (5): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (6): Conv1d(128, 1024, kernel_size=(1,), stride=(1,))
        (7): ReLU()
        (8): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (fcl): Sequential(
        (0): Linear(in_features=1024, out_features=512, bias=True)
        (1): ReLU()
        (2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (3): Linear(in_features=512, out_features=256, bias=True)
        (4): ReLU()
        (5): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_r

In [17]:
pointnet.train()

test_data = torch.rand(10, 3, num_points).to(device)
output, A = pointnet(test_data)
print(A.shape)
print(output.shape)

torch.Size([10, 64, 64])
torch.Size([10, 40])


In [8]:
num_epochs = 50
learning_rate = 0.0001
reg_weight = 0.001

optimizer = optim.Adam(pointnet.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=0.0001, max_lr=0.001, 
                                              step_size_up=2000, cycle_momentum=False)

criterion = PointNetClassificationLoss(reg_weight=reg_weight).to(device)

In [61]:
checkpoint = torch.load("pointnet_modelnet10.pth")
pointnet.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [9]:
directory = "./pointnet_modelnet40"
os.makedirs(directory, exist_ok=True)

for epoch in range(num_epochs):

    accuracy = 0
    loss_avg = 0
    count = 0

    pointnet.train()
    for data in trainloader:

        clouds = data.pos.view(data.batch[-1]+1, num_points, 3)
        clouds = clouds.transpose(2, 1).to(device)

        labels = data.y.to(device)

        optimizer.zero_grad()

        outputs, A = pointnet(clouds)
        loss = criterion(outputs, labels, A)
        loss.backward()
        optimizer.step()
        scheduler.step()

        loss_avg += loss.item()
        count += 1
    
    loss_avg = loss_avg/count
    
    pointnet.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for data in testloader:
            
            clouds = data.pos.view(data.batch[-1]+1, num_points, 3)
            clouds = clouds.transpose(2,1).to(device)

            labels = data.y.to(device)
            
            outputs, _ = pointnet(clouds)

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        accuracy = correct/total
    
    print("{}   [Epoch {:3}]  Loss: {:8.4}  Accuracy:   {:8.4}%".format(datetime.datetime.now(), epoch, loss_avg, 100*accuracy))

    torch.save(
        {'model_state_dict': pointnet.state_dict()},
        directory + "/epoch_" + str(epoch) + ".pth")

2025-03-05 00:39:30.791853   [Epoch   0]  Loss:    3.316  Accuracy:      24.88%
2025-03-05 00:40:27.214408   [Epoch   1]  Loss:    2.398  Accuracy:      38.21%
2025-03-05 00:41:23.821786   [Epoch   2]  Loss:     1.98  Accuracy:      47.16%
2025-03-05 00:42:20.551162   [Epoch   3]  Loss:    1.698  Accuracy:       53.2%
2025-03-05 00:43:17.495072   [Epoch   4]  Loss:    1.523  Accuracy:       59.4%
2025-03-05 00:44:14.215230   [Epoch   5]  Loss:    1.412  Accuracy:      59.81%
2025-03-05 00:45:10.971817   [Epoch   6]  Loss:    1.253  Accuracy:      59.89%
2025-03-05 00:46:07.847645   [Epoch   7]  Loss:    1.172  Accuracy:      59.28%
2025-03-05 00:47:04.528796   [Epoch   8]  Loss:    1.156  Accuracy:      49.23%
2025-03-05 00:48:01.280493   [Epoch   9]  Loss:    1.133  Accuracy:      60.09%
2025-03-05 00:48:58.082312   [Epoch  10]  Loss:    1.036  Accuracy:      63.33%
2025-03-05 00:49:55.552840   [Epoch  11]  Loss:    1.023  Accuracy:      69.89%
2025-03-05 00:50:52.406648   [Epoch  12]

In [66]:
pointnet.eval()

with torch.no_grad():
    correct = 0
    total = 0
    for data in testloader:
        
        clouds = data.pos.view(data.batch[-1]+1, num_points, 3)
        clouds = clouds.transpose(2, 1).to(device)

        labels = data.y.to(device)
            
        outputs, _ = pointnet(clouds)

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    accuracy = correct/total

    print("{}   Accuracy:   {:8.4}%".format(datetime.datetime.now(), 100*accuracy))

2025-03-05 10:58:18.512179   Accuracy:      90.42%


In [74]:
idx = random.randint(0, len(dataset_test))

data = dataset_test[idx]

cloud = data.pos.view(1, num_points, 3)
cloud = cloud.transpose(2, 1).to(device)

output, _ = pointnet(cloud)

probabilities = 100*F.softmax(output.transpose(1,0), dim=0)

_, predicted = torch.max(output.data, 1)
label = data.y

print('Predicted Class: {}    Certainty: {:8.4}   Actual Class:   {}'.format(classes[predicted.item()], probabilities[predicted.item()].item(), classes[label.item()]))

fig = go.Figure(
  data=[
    go.Scatter3d(
      x=data.pos[:,0], y=data.pos[:,1], z=data.pos[:,2],
      mode='markers',
      marker=dict(size=1, color="white"))],
  layout=dict(
    scene=dict(
      xaxis=dict(visible=False),
      yaxis=dict(visible=False),
      zaxis=dict(visible=False))))

fig.update_layout(template='plotly_dark')

fig.show()

Predicted Class: chair    Certainty:    99.99   Actual Class:   chair
